<a href="https://colab.research.google.com/github/Chumunaca/Chumunaca/blob/main/DURANIA_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Instalar dependencias básicas
!pip install -q streamlit pyngrok ultralytics timm joblib scikit-learn opencv-python-headless
!pip install -q streamlit pyngrok ultralytics timm joblib scikit-learn opencv-python-headless pyyaml==5.1 openpyxl
#Instalar Detectron2 (Truco para que jale en Colab)
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print(f"Instalando Detectron2 para Torch {TORCH_VERSION} y CUDA {CUDA_VERSION}...")

!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

print("Instalaciones completas.")

In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from PIL import Image
import torchvision.transforms as transforms
import timm
import joblib
import cv2
import os
import uuid
from datetime import datetime
import json
import io
import plotly.express as px
import plotly.graph_objects as go
from pyngrok import ngrok
import os
import time
import subprocess
#IMPORTACIONES DETECTRON2
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog


# CONFIGURACIÓN DE PÁGINA - TEMA OSCURO PROFESIONAL
st.set_page_config(
    page_title="Triage AI",
    page_icon="🏥",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Inicializar modo de vista
if 'view_mode' not in st.session_state:
    st.session_state.view_mode = 'doctor'  # 'doctor' o 'developer'

# ESTILO OSCURO PROFESIONAL CON ALTO CONTRASTE
st.markdown("""
<style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');

    * {
        font-family: 'Inter', sans-serif;
    }

    .stApp {
        background-color: #0a0a0a;
        color: #ffffff;
    }

    .main-header {
        font-size: 2.5rem;
        font-weight: 700;
        color: #ffffff;
        text-align: center;
        margin-bottom: 0.5rem;
        padding: 2rem 0 1rem;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
    }

    .subtitle {
        text-align: center;
        color: #e0e0e0;
        font-size: 1.1rem;
        margin-bottom: 3rem;
        font-weight: 400;
    }

    /* Cards con alto contraste */
    .card {
        background: #1a1a1a;
        border-radius: 12px;
        padding: 1.5rem;
        border: 1px solid #333333;
        box-shadow: 0 4px 20px rgba(0, 0, 0, 0.3);
        margin-bottom: 1rem;
        transition: all 0.3s ease;
    }

    .card:hover {
        border-color: #667eea;
        box-shadow: 0 6px 25px rgba(102, 126, 234, 0.2);
    }

    .risk-card-critical {
        border-left: 6px solid #ff4444;
        background: linear-gradient(135deg, #2a0f0f 0%, #1a1a1a 100%);
    }

    .risk-card-high {
        border-left: 6px solid #ffaa00;
        background: linear-gradient(135deg, #2a240f 0%, #1a1a1a 100%);
    }

    .risk-card-medium {
        border-left: 6px solid #0077ff;
        background: linear-gradient(135deg, #0f1a2a 0%, #1a1a1a 100%);
    }

    .risk-card-low {
        border-left: 6px solid #00cc66;
        background: linear-gradient(135deg, #0f2a1a 0%, #1a1a1a 100%);
    }

    /* Botones de alto contraste */
    .stButton > button {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border: none;
        border-radius: 8px;
        padding: 0.75rem 2rem;
        font-weight: 600;
        transition: all 0.3s ease;
        width: 100%;
    }

    .stButton > button:hover {
        transform: translateY(-2px);
        box-shadow: 0 6px 20px rgba(102, 126, 234, 0.4);
    }

    .secondary-button > button {
        background: #333333;
        color: #ffffff;
        border: 1px solid #555555;
    }

    /* Métricas con alto contraste */
    .metric-card {
        background: linear-gradient(135deg, #1a1a1a 0%, #2a2a2a 100%);
        border-radius: 12px;
        padding: 1.5rem;
        text-align: center;
        border: 1px solid #333333;
        box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2);
    }

    /* Pestañas contrastantes */
    .stTabs [data-baseweb="tab-list"] {
        gap: 1rem;
        background: #1a1a1a;
        border-radius: 10px;
        padding: 0.5rem;
        border: 1px solid #333333;
    }

    .stTabs [data-baseweb="tab"] {
        height: 45px;
        background: transparent;
        border-radius: 8px;
        padding: 0 1.5rem;
        font-weight: 600;
        color: #aaaaaa;
    }

    .stTabs [aria-selected="true"] {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: #ffffff;
    }

    /* Progress bars contrastantes */
    .progress-container {
        background: #333333;
        border-radius: 8px;
        height: 10px;
        margin: 1rem 0;
        overflow: hidden;
    }

    .progress-bar {
        height: 100%;
        border-radius: 8px;
        transition: width 0.6s ease;
    }

    /* Tags de hallazgos con contraste */
    .finding-tag {
        display: inline-block;
        background: rgba(59, 130, 246, 0.2);
        color: #60a5fa;
        padding: 0.4rem 1rem;
        border-radius: 16px;
        font-size: 0.8rem;
        margin: 0.2rem;
        border: 1px solid #3b82f6;
        font-weight: 600;
    }

    .critical-tag {
        background: rgba(239, 68, 68, 0.2);
        color: #f87171;
        border: 1px solid #ef4444;
    }

    /* Sidebar contrastante */
    .css-1d391kg, .css-1lcbm17 {
        background: #1a1a1a;
        border-right: 1px solid #333333;
    }

    /* Input fields contrastantes */
    .stTextInput > div > div > input,
    .stNumberInput > div > div > input,
    .stTextArea > div > div > textarea {
        background: #1a1a1a;
        border: 1px solid #333333;
        border-radius: 8px;
        color: #ffffff;
    }

    .stTextInput > div > div > input:focus,
    .stNumberInput > div > div > input:focus,
    .stTextArea > div > div > textarea:focus {
        border-color: #667eea;
        box-shadow: 0 0 0 2px rgba(102, 126, 234, 0.2);
    }

    /* Disclaimer contrastante */
    .disclaimer {
        background: #2a1a00;
        border: 1px solid #664400;
        border-radius: 8px;
        padding: 1.2rem;
        margin: 2rem 0 1rem;
        color: #ffaa00;
        font-size: 0.9rem;
        text-align: center;
        font-weight: 500;
    }

    /* Grid de imágenes */
    .image-grid {
        display: grid;
        grid-template-columns: repeat(2, 1fr);
        gap: 1rem;
        margin: 1rem 0;
    }

    .image-container {
        border: 1px solid #333333;
        border-radius: 8px;
        padding: 0.5rem;
        background: #1a1a1a;
    }

    .image-title {
        text-align: center;
        font-weight: 600;
        margin-bottom: 0.5rem;
        color: #e0e0e0;
    }

    /* Developer badge */
    .developer-badge {
        background: linear-gradient(135deg, #ff6b6b 0%, #ee5a52 100%);
        color: white;
        padding: 0.3rem 1rem;
        border-radius: 12px;
        font-size: 0.8rem;
        font-weight: 600;
        margin-left: 0.5rem;
    }

    .mode-indicator {
        display: inline-block;
        padding: 0.3rem 1rem;
        border-radius: 20px;
        font-size: 0.8rem;
        font-weight: 600;
        margin-left: 0.5rem;
    }

    .mode-doctor {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
    }

    .mode-developer {
        background: linear-gradient(135deg, #ff6b6b 0%, #ee5a52 100%);
        color: white;
    }
</style>
""", unsafe_allow_html=True)


# ARQUITECTURAS (MISMO CÓDIGO QUE SE USÓ CUANDO SE ENTRENABAN)

class DensityHybrid(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        self.cnn = timm.create_model('efficientnet_b3', pretrained=False)
        self.cnn.classifier = nn.Identity()
        self.age_net = nn.Sequential(nn.Linear(1, 16), nn.ReLU())
        self.head = nn.Sequential(
            nn.Linear(1536 + 16, 256),
            nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    def forward(self, img, age):
        x1 = self.cnn(img)
        x2 = self.age_net(age)
        return self.head(torch.cat([x1, x2], dim=1))

class BiradsModelPro(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = timm.create_model('tf_efficientnet_b7_ns', pretrained=False)
        self.cnn.classifier = nn.Identity()
        self.density_emb = nn.Embedding(4, 8)
        self.meta_net = nn.Sequential(nn.Linear(1 + 8, 32), nn.ReLU(), nn.BatchNorm1d(32))
        self.head = nn.Sequential(
            nn.Linear(2560 + 32, 512),
            nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
    def forward(self, img, age, dens):
        x_img = self.cnn(img)
        if age.dim() == 1: age = age.unsqueeze(1)
        x_meta = self.meta_net(torch.cat([age, self.density_emb(dens)], dim=1))
        return self.head(torch.cat([x_img, x_meta], dim=1))

# CARGA DE SISTEMA

@st.cache_resource
def load_system():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ROOT_DENS = '/content/drive/MyDrive/TESIS/density_model_final/'
    ROOT_BIRADS = '/content/drive/MyDrive/TESIS/birads_final_model/'
    ROOT_DET = '/content/drive/MyDrive/TESIS/detectron_output_vindr_A100_HEAVY/'
    DATA_POOL_DIR = '/content/drive/MyDrive/TESIS/live_data/'
    os.makedirs(DATA_POOL_DIR, exist_ok=True)

    try:
        dens = DensityHybrid(4).to(device)
        dens.load_state_dict(torch.load(os.path.join(ROOT_DENS, 'best_density_model.pth'), map_location=device))
        dens.eval()
        s_age_d = joblib.load(os.path.join(ROOT_DENS, 'age_scaler.pkl'))
        le_d = joblib.load(os.path.join(ROOT_DENS, 'density_classes.pkl'))

        birads = BiradsModelPro().to(device)
        birads.load_state_dict(torch.load(os.path.join(ROOT_BIRADS, 'best_birads_b7.pth'), map_location=device))
        birads.eval()
        s_age_b = joblib.load(os.path.join(ROOT_BIRADS, 'age_scaler_birads.pkl'))
        le_d_b = joblib.load(os.path.join(ROOT_BIRADS, 'density_encoder_birads.pkl'))

        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
        cfg.MODEL.WEIGHTS = os.path.join(ROOT_DET, 'model_final.pth')
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.35
        cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
        predictor = DefaultPredictor(cfg)

        vin_classes = ['Architectural Distortion', 'Asymmetry', 'Focal Asymmetry', 'Global Asymmetry', 'Mass', 'Nipple Retraction', 'Skin Retraction', 'Skin Thickening', 'Suspicious Calcification', 'Suspicious Lymph Node']

        return dens, s_age_d, le_d, birads, s_age_b, le_d_b, predictor, device, DATA_POOL_DIR, vin_classes
    except Exception as e:
        st.error(f"Error cargando modelos: {e}")
        return None

sys = load_system()

# BASE DE DATOS

DB_PATH = '/content/drive/MyDrive/TESIS/DATABASE_REENTRENAMIENTO.xlsx'

def save_feedback(patient_data, corrected_risk, corrected_density, notes, corrected_bool):
    record = {
        'Fecha': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        'ID_Paciente': patient_data['id'],
        'Edad': patient_data['age'],
        'AI_Riesgo_Score': round(patient_data['risk'], 4),
        'AI_Densidad': patient_data['density'],
        'AI_Hallazgos': ", ".join(patient_data['findings_text']),
        'AI_Coord_JSON': json.dumps(patient_data['box_data']),
        'Medico_Riesgo': corrected_risk,
        'Medico_Densidad': corrected_density,
        'Corregido_AI': "SÍ" if corrected_bool else "NO",
        'Notas': notes,
        'Rutas_Imagenes': str(patient_data['saved_paths'])
    }
    df_new = pd.DataFrame([record])

    if os.path.exists(DB_PATH):
        try:
            df_old = pd.read_excel(DB_PATH)
            df_final = pd.concat([df_old, df_new], ignore_index=True)
        except:
            df_final = df_new
    else:
        df_final = df_new

    df_final.to_excel(DB_PATH, index=False)
    return True

# LÓGICA DE PROCESAMIENTO

if 'queue' not in st.session_state:
    st.session_state.queue = []
if 'uploader_key' not in st.session_state:
    st.session_state.uploader_key = 0

def process_case(uploaded_files, age_val, sys_pack):
    dens_model, s_ad, le_d, birads_model, s_ab, le_db, predictor, device, save_dir, class_names = sys_pack

    patient_id = str(uuid.uuid4())[:8]
    max_risk = 0.0
    dominant_density_idx = 0
    saved_paths = []
    best_image_vis = None
    all_findings = set()
    all_boxes_data = []
    all_processed_images = []

    for i, up_file in enumerate(uploaded_files):
        save_path = os.path.join(save_dir, f"{patient_id}_img{i}.png")
        with open(save_path, "wb") as f:
            f.write(up_file.getbuffer())
        saved_paths.append(save_path)

        img_pil = Image.open(up_file).convert('RGB')
        img_np = np.array(img_pil)

        tfms_d = transforms.Compose([transforms.Resize((380, 380)), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
        tfms_b = transforms.Compose([transforms.Resize((512, 512)), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

        with torch.no_grad():
            age_t = torch.tensor(s_ad.transform([[age_val]]), dtype=torch.float32).to(device)
            out_d = dens_model(tfms_d(img_pil).unsqueeze(0).to(device), age_t)
            curr_dens_idx = torch.argmax(out_d, 1).item()
            if curr_dens_idx > dominant_density_idx: dominant_density_idx = curr_dens_idx

            curr_dens_label = le_d.inverse_transform([curr_dens_idx])[0]
            age_t_b = torch.tensor(s_ab.transform([[age_val]]), dtype=torch.float32).to(device)
            try: d_idx_b = le_db.transform([curr_dens_label])[0]
            except: d_idx_b = 0
            out_b = birads_model(tfms_b(img_pil).unsqueeze(0).to(device), age_t_b, torch.tensor([d_idx_b], dtype=torch.long).to(device))
            curr_risk = torch.sigmoid(out_b).item()

        outputs = predictor(img_np)
        instances = outputs["instances"].to("cpu")

        if len(instances.pred_boxes) > 0:
            boxes = instances.pred_boxes.tensor.numpy()
            classes_ids = instances.pred_classes.numpy()
            scores = instances.scores.numpy()
            for box, class_id, score in zip(boxes, classes_ids, scores):
                class_name = class_names[class_id]
                all_findings.add(class_name)
                all_boxes_data.append({
                    'class': class_name,
                    'score': float(score),
                    'box_xyxy': [float(x) for x in box],
                    'img_ref': f"img{i}.png"
                })

        v = Visualizer(img_np[:, :, ::-1], metadata=MetadataCatalog.get("vindr_train"), scale=1.0, instance_mode=ColorMode.IMAGE_BW)
        out_vis = v.draw_instance_predictions(instances)
        processed_img = out_vis.get_image()

        all_processed_images.append({
            'image': processed_img,
            'risk': curr_risk,
            'density': curr_dens_label,
            'index': i
        })

        if curr_risk > max_risk or best_image_vis is None:
            max_risk = curr_risk
            best_image_vis = processed_img

    final_density_label = le_d.inverse_transform([dominant_density_idx])[0]

    return {
        'id': patient_id, 'age': age_val, 'risk': max_risk, 'density': final_density_label,
        'vis_image': best_image_vis, 'saved_paths': saved_paths, 'num_images': len(uploaded_files),
        'findings_text': list(all_findings) if all_findings else ["Sin hallazgos específicos"],
        'box_data': all_boxes_data,
        'all_processed_images': all_processed_images
    }


# FUNCIONES DE VISUALIZACIÓN

def create_risk_distribution_chart(queue):
    if not queue:
        return None

    risks = [p['risk'] for p in queue]
    risk_levels = ['Bajo' if r < 0.2 else 'Medio' if r < 0.4 else 'Alto' if r < 0.6 else 'Crítico' for r in risks]

    fig = px.pie(
        names=risk_levels,
        title="Distribución de Niveles de Riesgo",
        color=risk_levels,
        color_discrete_map={
            'Bajo': '#00cc66',
            'Medio': '#0077ff',
            'Alto': '#ffaa00',
            'Crítico': '#ff4444'
        }
    )
    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        font_color='#ffffff',
        showlegend=True
    )
    return fig

def create_performance_metrics(df):
    if df.empty:
        return None

    accuracy = len(df[df['Corregido_AI'] == 'NO']) / len(df) * 100 if len(df) > 0 else 0

    fig = go.Figure()

    fig.add_trace(go.Indicator(
        mode="gauge+number",
        value=accuracy,
        domain={'x': [0, 1], 'y': [0, 1]},
        title={'text': "Precisión del Sistema"},
        gauge={
            'axis': {'range': [None, 100]},
            'bar': {'color': "#667eea"},
            'steps': [
                {'range': [0, 50], 'color': "lightgray"},
                {'range': [50, 80], 'color': "gray"},
            ],
            'threshold': {
                'line': {'color': "red", 'width': 4},
                'thickness': 0.75,
                'value': 90
            }
        }
    ))

    fig.update_layout(
        paper_bgcolor='rgba(0,0,0,0)',
        font={'color': "#ffffff", 'family': "Inter"}
    )

    return fig

# INTERFAZ PRINCIPAL

st.markdown("<h1 class='main-header'>Triage de Mamografías asistidas por IA</h1>", unsafe_allow_html=True)

# Header con indicador de modo
current_mode = st.session_state.view_mode
mode_badge = "developer" if current_mode == "developer" else "doctor"
mode_text = "MODO DESARROLLADOR" if current_mode == "developer" else "MODO MÉDICO"

st.markdown(f"<p class='subtitle'>Sistema de apoyo para clasificación de mamografías según el sistema BI-RADS <span class='mode-indicator mode-{mode_badge}'>{mode_text}</span></p>", unsafe_allow_html=True)

# Sidebar profesional
with st.sidebar:
    # Selector de modo - DIRECTAMENTE AL GRANO
    st.markdown("<div class='card'>", unsafe_allow_html=True)
    st.markdown("### Tipo de Visualización")

    col1, col2 = st.columns(2)
    with col1:
        if st.button("👨‍⚕️ Médico", use_container_width=True, type="primary" if st.session_state.view_mode == 'doctor' else "secondary"):
            st.session_state.view_mode = 'doctor'
            st.rerun()
    with col2:
        if st.button("👨‍💻 Desarrollador", use_container_width=True, type="primary" if st.session_state.view_mode == 'developer' else "secondary"):
            st.session_state.view_mode = 'developer'
            st.rerun()

    st.markdown("**Tipo actual:**")
    if st.session_state.view_mode == 'doctor':
        st.success("👨‍⚕️ Vista médica")
    else:
        st.warning("👨‍💻 Vista desarrollador")

    st.markdown("</div>", unsafe_allow_html=True)

    st.markdown("<div class='card'>", unsafe_allow_html=True)
    st.markdown("### Nuevo Estudio")

    with st.form("ingreso", clear_on_submit=True):
        files = st.file_uploader("Subir mamografías", type=['png', 'jpg', 'jpeg'],
                               accept_multiple_files=True,
                               key=f"uploader_{st.session_state.uploader_key}",
                               help="Máximo 4 imágenes por estudio, por favor")

        age = st.number_input("Edad", min_value=18, max_value=100, value=50)

        submitted = st.form_submit_button("Analizar", use_container_width=True)

        if submitted and files and sys:
            with st.spinner("Procesando estudio mamográfico..."):
                case = process_case(files, age, sys)
                st.session_state.queue.append(case)
                st.session_state.queue.sort(key=lambda x: x['risk'], reverse=True)
                st.session_state.uploader_key += 1
                st.success(f"Estudio {case['id']} procesado")
                st.rerun()

    st.markdown("</div>", unsafe_allow_html=True)

    # Estadísticas rápidas
    if st.session_state.queue:
        total = len(st.session_state.queue)
        critical = sum(1 for p in st.session_state.queue if p['risk'] > 0.6)
        high = sum(1 for p in st.session_state.queue if 0.4 < p['risk'] <= 0.6)

        st.markdown("<div class='card'>", unsafe_allow_html=True)
        st.markdown("### Resumen")
        st.markdown(f"**Total:** {total}")
        st.markdown(f"**Críticos:** {critical}")
        st.markdown(f"**Alto Riesgo:** {high}")
        st.markdown("</div>", unsafe_allow_html=True)


# VISTAS SEGÚN MODO


if st.session_state.view_mode == 'doctor':
    # MODO MÉDICO - Solo Dashboard Principal
    st.markdown("### 🏥 Panel de Triage")
    st.markdown("Interfaz optimizada para priorizar estudios")

    if not st.session_state.queue:
        col1, col2, col3 = st.columns([1, 2, 1])
        with col2:
            st.markdown("<div class='card' style='text-align: center; padding: 4rem;'>", unsafe_allow_html=True)
            st.markdown("<h3 style='color: inherit;'>No se ha subido ningún estudio</h3>", unsafe_allow_html=True)
            st.markdown("<p style='color: inherit; opacity: 0.7;'>Suba un estudio mamográfico para comenzar el análisis</p>", unsafe_allow_html=True)
            st.markdown("</div>", unsafe_allow_html=True)
    else:
        q = st.session_state.queue
        critical_count = sum(1 for p in q if p['risk'] > 0.6)
        high_count = sum(1 for p in q if 0.4 < p['risk'] <= 0.6)
        medium_count = sum(1 for p in q if 0.2 <= p['risk'] <= 0.4)
        low_count = sum(1 for p in q if p['risk'] < 0.2)

        # Métricas principales
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
            st.markdown(f"<h2 style='margin: 0;'>{len(q)}</h2>", unsafe_allow_html=True)
            st.markdown("<p style='margin: 0; opacity: 0.8;'>Total en espera</p>", unsafe_allow_html=True)
            st.markdown("</div>", unsafe_allow_html=True)

        with col2:
            st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
            st.markdown(f"<h2 style='color: #ff4444; margin: 0;'>{critical_count}</h2>", unsafe_allow_html=True)
            st.markdown("<p style='margin: 0; opacity: 0.8;'>Casos críticos</p>", unsafe_allow_html=True)
            st.markdown("</div>", unsafe_allow_html=True)

        with col3:
            st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
            st.markdown(f"<h2 style='color: #ffaa00; margin: 0;'>{high_count}</h2>", unsafe_allow_html=True)
            st.markdown("<p style='margin: 0; opacity: 0.8;'>Alto riesgo</p>", unsafe_allow_html=True)
            st.markdown("</div>", unsafe_allow_html=True)

        with col4:
            st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
            st.markdown(f"<h2 style='color: #00cc66; margin: 0;'>{low_count + medium_count}</h2>", unsafe_allow_html=True)
            st.markdown("<p style='margin: 0; opacity: 0.8;'>Bajo/medio riesgo</p>", unsafe_allow_html=True)
            st.markdown("</div>", unsafe_allow_html=True)

        st.markdown("---")

        # Lista de pacientes
        for i, patient in enumerate(q):
            risk_score = patient['risk']

            if risk_score > 0.6:
                risk_class = "risk-card-critical"
                risk_label = "CRÍTICO"
                priority_color = "#ff4444"
                priority_icon = "🚨"
            elif risk_score > 0.4:
                risk_class = "risk-card-high"
                risk_label = "ALTO RIESGO"
                priority_color = "#ffaa00"
                priority_icon = "⚠️"
            elif risk_score >= 0.2:
                risk_class = "risk-card-medium"
                risk_label = "RIESGO MEDIO"
                priority_color = "#0077ff"
                priority_icon = "📊"
            else:
                risk_class = "risk-card-low"
                risk_label = "BAJO RIESGO"
                priority_color = "#00cc66"
                priority_icon = "✅"

            with st.expander(f"{priority_icon} Paciente {patient['id'].upper()} • {patient['age']} años • Riesgo: {risk_score:.1%} • {risk_label}", expanded=risk_score > 0.4):
                st.markdown(f"<div class='{risk_class}'>", unsafe_allow_html=True)

                col1, col2 = st.columns([2, 1])

                with col1:
                    st.image(patient['vis_image'], use_container_width=True, caption="Imagen con mayor riesgo - Análisis con detecciones automáticas")

                    if 'all_processed_images' in patient and len(patient['all_processed_images']) > 1:
                        st.markdown("---")
                        st.markdown("### Todas las imágenes analizadas")
                        st.markdown("<div class='image-grid'>", unsafe_allow_html=True)

                        cols = st.columns(2)
                        for idx, img_data in enumerate(patient['all_processed_images']):
                            with cols[idx % 2]:
                                st.markdown(f"<div class='image-container'>", unsafe_allow_html=True)
                                st.markdown(f"<div class='image-title'>Imagen {idx + 1}</div>", unsafe_allow_html=True)
                                st.image(img_data['image'], use_container_width=True,
                                       caption=f"Riesgo: {img_data['risk']:.1%} • Densidad: {img_data['density']}")
                                st.markdown("</div>", unsafe_allow_html=True)

                        st.markdown("</div>", unsafe_allow_html=True)

                    st.markdown("**Hallazgos detectados:**")
                    if patient['findings_text'] == ["Sin hallazgos específicos"]:
                        st.info("Sin hallazgos patológicos detectados")
                    else:
                        for finding in patient['findings_text']:
                            is_critical = any(x in finding.lower() for x in ['suspicious', 'mass', 'calcification'])
                            tag_class = "finding-tag critical-tag" if is_critical else "finding-tag"
                            st.markdown(f"<span class='{tag_class}'>{finding}</span>", unsafe_allow_html=True)

                with col2:
                    st.markdown("<div class='card'>", unsafe_allow_html=True)
                    st.markdown("### Información del Paciente")
                    st.markdown(f"**ID:** {patient['id']}")
                    st.markdown(f"**Edad:** {patient['age']} años")
                    st.markdown(f"**Densidad:** {patient['density']}")
                    st.markdown(f"**Imágenes:** {patient['num_images']}")

                    st.markdown("**Nivel de riesgo:**")
                    st.markdown(f"""
                    <div class="progress-container">
                        <div class="progress-bar" style="width: {risk_score*100}%; background: {priority_color};"></div>
                    </div>
                    <p style="text-align: center; margin: 0; font-weight: 600; color: {priority_color};">{risk_score:.1%}</p>
                    """, unsafe_allow_html=True)

                    st.markdown("</div>", unsafe_allow_html=True)

                    st.markdown("<div class='card'>", unsafe_allow_html=True)
                    st.markdown("### Validación Médica")

                    with st.form(key=f"val_{patient['id']}"):
                        corrected_risk = st.radio(
                            "Diagnóstico real:",
                            ["Bajo Riesgo", "Alto Riesgo"],
                            index=1 if risk_score > 0.4 else 0
                        )

                        corrected_density = st.selectbox(
                            "Densidad real:",
                            ["DENSITY A", "DENSITY B", "DENSITY C", "DENSITY D"],
                            index=["DENSITY A", "DENSITY B", "DENSITY C", "DENSITY D"].index(patient['density'])
                        )

                        notes = st.text_area("Notas clínicas:", placeholder="Observaciones, diagnóstico BI-RADS, comentarios...")

                        col_btn1, col_btn2 = st.columns(2)
                        with col_btn1:
                            if st.form_submit_button("Guardar Corrección", use_container_width=True):
                                ai_risk_bool = risk_score > 0.4
                                doc_risk_bool = (corrected_risk == "Alto Riesgo")
                                corrected = (ai_risk_bool != doc_risk_bool) or (patient['density'] != corrected_density)

                                save_feedback(patient, corrected_risk, corrected_density, notes, corrected)
                                st.success("Corrección guardada para retroalimentar el modelo")

                        with col_btn2:
                            if st.form_submit_button("Marcar como atendido", use_container_width=True):
                                st.session_state.queue = [case for case in st.session_state.queue if case['id'] != patient['id']]
                                st.rerun()

                    st.markdown("</div>", unsafe_allow_html=True)

                st.markdown("</div>", unsafe_allow_html=True)

else:
    # MODO DESARROLLADOR
    tab1, tab2, tab3 = st.tabs(["📊 Página principal", "📈 Análisis", "💾 Base de Datos"])

    with tab1:
        st.markdown("### 🖥️ Panel de Control")
        st.markdown("Vista completa con todas las funcionalidades del sistema")

        if not st.session_state.queue:
            col1, col2, col3 = st.columns([1, 2, 1])
            with col2:
                st.markdown("<div class='card' style='text-align: center; padding: 4rem;'>", unsafe_allow_html=True)
                st.markdown("<h3 style='color: inherit;'>Sala de espera vacía</h3>", unsafe_allow_html=True)
                st.markdown("<p style='color: inherit; opacity: 0.7;'>Suba un estudio mamográfico para comenzar el análisis</p>", unsafe_allow_html=True)
                st.markdown("</div>", unsafe_allow_html=True)
        else:
            q = st.session_state.queue
            critical_count = sum(1 for p in q if p['risk'] > 0.6)
            high_count = sum(1 for p in q if 0.4 < p['risk'] <= 0.6)
            medium_count = sum(1 for p in q if 0.2 <= p['risk'] <= 0.4)
            low_count = sum(1 for p in q if p['risk'] < 0.2)

            col1, col2, col3, col4 = st.columns(4)
            with col1:
                st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
                st.markdown(f"<h2 style='margin: 0;'>{len(q)}</h2>", unsafe_allow_html=True)
                st.markdown("<p style='margin: 0; opacity: 0.8;'>Total en espera</p>", unsafe_allow_html=True)
                st.markdown("</div>", unsafe_allow_html=True)

            with col2:
                st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
                st.markdown(f"<h2 style='color: #ff4444; margin: 0;'>{critical_count}</h2>", unsafe_allow_html=True)
                st.markdown("<p style='margin: 0; opacity: 0.8;'>Casos críticos</p>", unsafe_allow_html=True)
                st.markdown("</div>", unsafe_allow_html=True)

            with col3:
                st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
                st.markdown(f"<h2 style='color: #ffaa00; margin: 0;'>{high_count}</h2>", unsafe_allow_html=True)
                st.markdown("<p style='margin: 0; opacity: 0.8;'>Alto riesgo</p>", unsafe_allow_html=True)
                st.markdown("</div>", unsafe_allow_html=True)

            with col4:
                st.markdown("<div class='metric-card'>", unsafe_allow_html=True)
                st.markdown(f"<h2 style='color: #00cc66; margin: 0;'>{low_count + medium_count}</h2>", unsafe_allow_html=True)
                st.markdown("<p style='margin: 0; opacity: 0.8;'>Bajo/medio riesgo</p>", unsafe_allow_html=True)
                st.markdown("</div>", unsafe_allow_html=True)

            # Gráfico de distribución
            col1, col2 = st.columns([2, 1])
            with col1:
                fig = create_risk_distribution_chart(q)
                if fig:
                    st.plotly_chart(fig, use_container_width=True)

            with col2:
                st.markdown("<div class='card'>", unsafe_allow_html=True)
                st.markdown("### 🚨 Prioridades")
                if critical_count > 0:
                    st.error(f"{critical_count} casos críticos requieren atención inmediata")
                if high_count > 0:
                    st.warning(f"{high_count} casos de alto riesgo necesitan revisión prioritaria")
                st.markdown("</div>", unsafe_allow_html=True)

            st.markdown("---")

            for i, patient in enumerate(q):
                risk_score = patient['risk']

                if risk_score > 0.6:
                    risk_class = "risk-card-critical"
                    risk_label = "CRÍTICO"
                    priority_color = "#ff4444"
                    priority_icon = "🚨"
                elif risk_score > 0.4:
                    risk_class = "risk-card-high"
                    risk_label = "ALTO RIESGO"
                    priority_color = "#ffaa00"
                    priority_icon = "⚠️"
                elif risk_score >= 0.2:
                    risk_class = "risk-card-medium"
                    risk_label = "RIESGO MEDIO"
                    priority_color = "#0077ff"
                    priority_icon = "📊"
                else:
                    risk_class = "risk-card-low"
                    risk_label = "BAJO RIESGO"
                    priority_color = "#00cc66"
                    priority_icon = "✅"

                with st.expander(f"{priority_icon} Paciente {patient['id'].upper()} • {patient['age']} años • Riesgo: {risk_score:.1%} • {risk_label}", expanded=risk_score > 0.4):
                    st.markdown(f"<div class='{risk_class}'>", unsafe_allow_html=True)

                    col1, col2 = st.columns([2, 1])

                    with col1:
                        st.image(patient['vis_image'], use_container_width=True, caption="Imagen con mayor riesgo - Análisis con detecciones automáticas")

                        if 'all_processed_images' in patient and len(patient['all_processed_images']) > 1:
                            st.markdown("---")
                            st.markdown("### Todas las imágenes analizadas")
                            st.markdown("<div class='image-grid'>", unsafe_allow_html=True)

                            cols = st.columns(2)
                            for idx, img_data in enumerate(patient['all_processed_images']):
                                with cols[idx % 2]:
                                    st.markdown(f"<div class='image-container'>", unsafe_allow_html=True)
                                    st.markdown(f"<div class='image-title'>Imagen {idx + 1}</div>", unsafe_allow_html=True)
                                    st.image(img_data['image'], use_container_width=True,
                                           caption=f"Riesgo: {img_data['risk']:.1%} • Densidad: {img_data['density']}")
                                    st.markdown("</div>", unsafe_allow_html=True)

                            st.markdown("</div>", unsafe_allow_html=True)

                        st.markdown("**Hallazgos detectados:**")
                        if patient['findings_text'] == ["Sin hallazgos específicos"]:
                            st.info("Sin hallazgos patológicos detectados")
                        else:
                            for finding in patient['findings_text']:
                                is_critical = any(x in finding.lower() for x in ['suspicious', 'mass', 'calcification'])
                                tag_class = "finding-tag critical-tag" if is_critical else "finding-tag"
                                st.markdown(f"<span class='{tag_class}'>{finding}</span>", unsafe_allow_html=True)

                    with col2:
                        st.markdown("<div class='card'>", unsafe_allow_html=True)
                        st.markdown("### Información del Paciente")
                        st.markdown(f"**ID:** {patient['id']}")
                        st.markdown(f"**Edad:** {patient['age']} años")
                        st.markdown(f"**Densidad:** {patient['density']}")
                        st.markdown(f"**Imágenes:** {patient['num_images']}")

                        st.markdown("**Nivel de riesgo:**")
                        st.markdown(f"""
                        <div class="progress-container">
                            <div class="progress-bar" style="width: {risk_score*100}%; background: {priority_color};"></div>
                        </div>
                        <p style="text-align: center; margin: 0; font-weight: 600; color: {priority_color};">{risk_score:.1%}</p>
                        """, unsafe_allow_html=True)

                        st.markdown("</div>", unsafe_allow_html=True)

                        st.markdown("<div class='card'>", unsafe_allow_html=True)
                        st.markdown("### Validación Médica")

                        with st.form(key=f"val_{patient['id']}"):
                            corrected_risk = st.radio(
                                "Diagnóstico real:",
                                ["Bajo Riesgo", "Alto Riesgo"],
                                index=1 if risk_score > 0.4 else 0
                            )

                            corrected_density = st.selectbox(
                                "Densidad real:",
                                ["DENSITY A", "DENSITY B", "DENSITY C", "DENSITY D"],
                                index=["DENSITY A", "DENSITY B", "DENSITY C", "DENSITY D"].index(patient['density'])
                            )

                            notes = st.text_area("Notas clínicas:", placeholder="Observaciones, diagnóstico BI-RADS, comentarios...")

                            col_btn1, col_btn2 = st.columns(2)
                            with col_btn1:
                                if st.form_submit_button("Guardar Feedback", use_container_width=True):
                                    ai_risk_bool = risk_score > 0.4
                                    doc_risk_bool = (corrected_risk == "Alto Riesgo")
                                    corrected = (ai_risk_bool != doc_risk_bool) or (patient['density'] != corrected_density)

                                    save_feedback(patient, corrected_risk, corrected_density, notes, corrected)
                                    st.success("Feedback guardado para aprendizaje activo")

                            with col_btn2:
                                if st.form_submit_button("Marcar Atendido", use_container_width=True):
                                    st.session_state.queue = [case for case in st.session_state.queue if case['id'] != patient['id']]
                                    st.rerun()

                        st.markdown("</div>", unsafe_allow_html=True)

                    st.markdown("</div>", unsafe_allow_html=True)

    with tab2:
        st.markdown("<div class='card'>", unsafe_allow_html=True)
        st.markdown("<h2>📈 Análisis & Reportes</h2>", unsafe_allow_html=True)
        st.markdown("Panel de métricas y análisis del sistema")

        if os.path.exists(DB_PATH):
            df = pd.read_excel(DB_PATH)

            if not df.empty:
                col1, col2 = st.columns(2)

                with col1:
                    # Métricas de performance
                    total_cases = len(df)
                    corrected_cases = len(df[df['Corregido_AI'] == 'SÍ'])
                    accuracy = ((total_cases - corrected_cases) / total_cases * 100) if total_cases > 0 else 0

                    st.metric("Total de Casos", total_cases)
                    st.metric("Casos Corregidos", corrected_cases)
                    st.metric("Precisión del Sistema", f"{accuracy:.1f}%")

                with col2:
                    fig = create_performance_metrics(df)
                    if fig:
                        st.plotly_chart(fig, use_container_width=True)

                # Gráficos adicionales
                col1, col2 = st.columns(2)

                with col1:
                    # Distribución de densidades
                    density_counts = df['AI_Densidad'].value_counts()
                    fig_density = px.bar(
                        x=density_counts.index,
                        y=density_counts.values,
                        title="Distribución de Densidades Mamarias",
                        color_discrete_sequence=['#667eea']
                    )
                    fig_density.update_layout(
                        paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)',
                        font_color='#ffffff',
                        showlegend=False
                    )
                    st.plotly_chart(fig_density, use_container_width=True)

                with col2:
                    # Distribución por edad
                    fig_age = px.histogram(
                        df,
                        x='Edad',
                        title="Distribución de Edades",
                        nbins=20,
                        color_discrete_sequence=['#667eea']
                    )
                    fig_age.update_layout(
                        paper_bgcolor='rgba(0,0,0,0)',
                        plot_bgcolor='rgba(0,0,0,0)',
                        font_color='#ffffff'
                    )
                    st.plotly_chart(fig_age, use_container_width=True)
            else:
                st.info("No hay datos suficientes para generar reportes")
        else:
            st.info("La base de datos se creará automáticamente con el primer registro")

        st.markdown("</div>", unsafe_allow_html=True)

    with tab3:
        st.markdown("<div class='card'>", unsafe_allow_html=True)
        st.markdown("<h2>💾 Base de Datos y Exportación</h2>", unsafe_allow_html=True)
        st.markdown("Gestión completa de datos y exportación")

        if os.path.exists(DB_PATH):
            df = pd.read_excel(DB_PATH)

            st.dataframe(df, use_container_width=True)

            # Estadísticas de la base de datos
            col1, col2, col3 = st.columns(3)
            with col1:
                st.metric("Registros Totales", len(df))
            with col2:
                st.metric("Correcciones Realizadas", len(df[df['Corregido_AI'] == 'SÍ']))
            with col3:
                st.metric("Precisión Actual", f"{(len(df[df['Corregido_AI'] == 'NO']) / len(df) * 100):.1f}%" if len(df) > 0 else "0%")

            # Opciones de exportación
            st.markdown("### Exportar Datos")

            col1, col2 = st.columns(2)

            with col1:
                # Exportar a Excel
                def to_excel(df):
                    output = io.BytesIO()
                    with pd.ExcelWriter(output, engine='openpyxl') as writer:
                        df.to_excel(writer, index=False, sheet_name='Datos_Triage')
                    return output.getvalue()

                excel_data = to_excel(df)
                st.download_button(
                    label="📊 Descargar Excel",
                    data=excel_data,
                    file_name=f"triage_database_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx",
                    mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
                    use_container_width=True
                )

            with col2:
                # Exportar a CSV
                csv_data = df.to_csv(index=False).encode('utf-8')
                st.download_button(
                    label="📄 Descargar CSV",
                    data=csv_data,
                    file_name=f"triage_database_{datetime.now().strftime('%Y%m%d_%H%M')}.csv",
                    mime="text/csv",
                    use_container_width=True
                )
        else:
            st.info("La base de datos se creará automáticamente con el primer registro")

        st.markdown("</div>", unsafe_allow_html=True)

# Disclaimer en la parte inferior
st.markdown("---")
st.markdown("""
<div class='disclaimer'>
    <strong>Nota importante:</strong> Esta aplicación se encuentra en fase de prueba y además no pretende sustituir al diagnóstico de un profesional médico.
</div>
""", unsafe_allow_html=True)

In [ ]:

ngrok.kill()
os.system("killall -9 streamlit")
# TOKEN
!ngrok config add-authtoken "AQUI VA MI TOKEN:)"
print("Lanzando app")
subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501"])
time.sleep(3)

public_url = ngrok.connect(8501)
print(f"DASHBOARD: {public_url}")